# PageRank

> PageRank 是网页排名的一种算法，也是google搜索的核心算法。属于图数据上的无监督学习方法。PageRank是有向图上的定义的一个随机游走模型，即一阶马尔科夫链，描述每一个节点在稳定状态下的概率，该概率就是每一个节点的PageRank值。

这里定义一个简单的有向图的模型。

In [1]:
import torch

In [29]:
# 定义每一个节点的转移矩阵
M = torch.tensor([[0,   1/2, 1,  0],
                  [1/3,  0,  0,  1/2],
                  [1/3,  0,  0,  1/2],
                  [1/3, 1/2, 0,  0],
])

> 由于是概率图，因此需要满足和为1的性质。

In [30]:
torch.sum(M, dim=0)

tensor([1., 1., 1., 1.])

In [31]:
def PageRank(R0, M, iters):
    for _ in range(iters):
        R0 = torch.matmul(M, R0)
    return R0

In [32]:
# 开始的时候，设置每一个节点的概率一样。
R0 = torch.ones((4,1)).fill_(1/4)
PageRank(R0, M, 1000000)

tensor([[0.3333],
        [0.2222],
        [0.2222],
        [0.2222]])

> 我们得到了每一个节点的稳定值.

## PageRank的一般定义

由于有些网页无法调到其他网页，因此基本的PageRank算法是不适用的。这里需要加上一个平滑项。

$ R = dMR + \frac{1-d}{n} 1$

其中d为阻尼因子，$1$是分量为1的n维向量。

In [38]:
def PageRank(R0, d, M, iters):
    dM = d * M
    smooth = (1-d) * torch.ones((len(M),1)).fill_(1/len(M))
    
    for _ in range(iters):
        R0 = torch.matmul(dM, R0) + smooth
    return R0

In [39]:
M = torch.tensor([[0,   1/2, 0,  0],
                  [1/3,  0,  0,  1/2],
                  [1/3,  0,  1,  1/2],
                  [1/3, 1/2, 0,  0],
])

R0 = torch.ones((4,1)).fill_(1/4)
PageRank(R0, 0.8, M, 100000)

tensor([[0.1014],
        [0.1284],
        [0.6419],
        [0.1284]])

# 幂法来计算PageRank

思想是使用特征值与特征向量计算。

In [84]:
M = torch.tensor([[0,   0, 1],
                  [1/2, 0, 0],
                  [1/2, 1, 0],
])

x = torch.ones((3,1))

A = 0.85 * M + (1-0.85) / 3 * torch.ones((3,3))

# 这里采用的范数是无穷范数,即分量中绝对值的最大值。
def get_norm(A):
    A = torch.abs(A)
    return torch.max(A)

In [72]:
y = A @ x
x = y / get_norm(y)

In [81]:
for _ in range(100):
    t = x
    y = A @ x
    x = y / get_norm(y)
    
    if torch.norm(x-t, 2) < 0.0000001:
        break
print(x)

tensor([[0.9758],
        [0.5405],
        [1.0000]])


In [83]:
print(x / torch.sum(x))

tensor([[0.3878],
        [0.2148],
        [0.3974]])


## 逆矩阵

In [65]:
M = torch.tensor([[0,   0, 1],
                  [1/2, 0, 0],
                  [1/2, 1, 0],
])

In [70]:
torch.inverse(torch.eye(3) - 0.85*M) @ ((1-0.85) / 3 * torch.ones((3,1)))

tensor([[0.3878],
        [0.2148],
        [0.3974]])